In [1]:
import json

with open("data/world_happiness_data.json", "r") as f:
    data = json.load(f)

data_23 = {}
for country in data:
    for year in data[country]:
        if year == "2023":
            data_23[country] = data[country][year]